# Adjust this to copy files from G Drive to the USB-C Drive without errors

In [ ]:
"""Google Colab setup: mount Drive and install package dependencies.

Run this cell first. If prompted to restart the runtime, do so, then re-run all cells.
This cell is a no-op when running locally.
"""
import sys, os
if 'google.colab' in sys.modules:
    from google.colab import drive
    drive.mount('/content/drive')

    NOTEBOOK_DIR = '/content/drive/My Drive/Colab Notebooks/ei_notebooks/research/reproducible_research_2026_02_02_basic_volatility'

    os.chdir(NOTEBOOK_DIR)
    if NOTEBOOK_DIR not in sys.path:
        sys.path.append(NOTEBOOK_DIR)

    !pip install ergodic-insurance -q 2>&1 | tail -3
    print('\nSetup complete. If you see numpy/scipy import errors below,')
    print('restart the runtime (Runtime > Restart runtime) and re-run all cells.')

In [ ]:
import shutil
import os
from pathlib import Path
import re

def sanitize_filename(filename):
    """Remove or replace invalid characters for Windows/USB drives"""
    # Replace problematic characters
    invalid_chars = r'[<>:"|?*]'
    filename = re.sub(invalid_chars, '_', filename)
    # Remove leading/trailing spaces and dots
    filename = filename.strip('. ')
    return filename

def copy_files_safe(source_dir, dest_dir, max_retries=3):
    """Copy files with error handling and retry logic"""
    source_path = Path(source_dir)
    dest_path = Path(dest_dir)
    
    # Create destination if it doesn't exist
    dest_path.mkdir(parents=True, exist_ok=True)
    
    copied = []
    failed = []
    skipped = []
    
    for item in source_path.rglob('*'):
        if item.is_file():
            try:
                # Calculate relative path
                rel_path = item.relative_to(source_path)
                
                # Sanitize each part of the path
                sanitized_parts = [sanitize_filename(part) for part in rel_path.parts]
                dest_file = dest_path / Path(*sanitized_parts)
                
                # Check if file already exists
                if dest_file.exists():
                    skipped.append(str(rel_path))
                    # print(f"⊘ Skipped (exists): {rel_path}")
                    continue
                
                # Create parent directories
                dest_file.parent.mkdir(parents=True, exist_ok=True)
                
                # Copy with retry
                for attempt in range(max_retries):
                    try:
                        shutil.copy2(item, dest_file)
                        copied.append(str(rel_path))
                        print(f"✓ Copied: {rel_path}")
                        break
                    except Exception as e:
                        if attempt == max_retries - 1:
                            raise
                        print(f"  Retry {attempt + 1} for {rel_path}")
                        
            except Exception as e:
                failed.append((str(item), str(e)))
                print(f"✗ Failed: {item} - {e}")
    
    print(f"\n{len(copied)} files copied successfully")
    print(f"{len(skipped)} files skipped (already exist)")
    if failed:
        print(f"{len(failed)} files failed:")
        for path, error in failed:
            print(f"  {path}: {error}")
    
    return copied, failed, skipped

### Small Model #####
# source = r"G:\My Drive\Colab Notebooks\results"  # Google Drive
# destination = r"D:\2026-01-29 Vol Small"  # USB drive

### Full Volatility Model #####
source = r"G:\My Drive\Colab Notebooks\results_vol_full"  # Google Drive
destination = r"D:\2026-01-31 Vol"  # USB drive

copy_files_safe(source, destination, max_retries=10)